# AlongTrack Data

In [1]:
import autoroot
import typing as tp
from dataclasses import dataclass
import functools as ft
import numpy as np
import pandas as pd
import xarray as xr
import einops
from metpy.units import units
import pint_xarray
import xarray_dataclasses as xrdataclass
from oceanbench._src.datasets.base import XRDABatcher
from oceanbench._src.geoprocessing.spatial import transform_360_to_180
from oceanbench._src.geoprocessing.subset import where_slice
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import seaborn as sns

sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)

%load_ext autoreload
%autoreload 2


## Data

In [2]:
!ls "/gpfswork/rech/yrf/commun/data_challenges/dc20a_osse/raw/dc_obs/"

2020a_SSH_mapping_NATL60_envisat.nc
2020a_SSH_mapping_NATL60_geosat2.nc
2020a_SSH_mapping_NATL60_jason1.nc
2020a_SSH_mapping_NATL60_karin_swot.nc
2020a_SSH_mapping_NATL60_nadir_swot.nc
2020a_SSH_mapping_NATL60_topex-poseidon_interleaved.nc


In [3]:

!ls /gpfswork/rech/yrf/commun/data_challenges/dc21a_ose/test/train

dt_gulfstream_alg_phy_l3_20161201-20180131_285-315_23-53.nc
dt_gulfstream_h2g_phy_l3_20161201-20180131_285-315_23-53.nc
dt_gulfstream_j2g_phy_l3_20161201-20180131_285-315_23-53.nc
dt_gulfstream_j2n_phy_l3_20161201-20180131_285-315_23-53.nc
dt_gulfstream_j3_phy_l3_20161201-20180131_285-315_23-53.nc
dt_gulfstream_s3a_phy_l3_20161201-20180131_285-315_23-53.nc


In [4]:
files_nadir_dc20a = [
    "/gpfswork/rech/yrf/commun/data_challenges/dc20a_osse/raw/dc_obs/2020a_SSH_mapping_NATL60_jason1.nc",
    "/gpfswork/rech/yrf/commun/data_challenges/dc20a_osse/raw/dc_obs/2020a_SSH_mapping_NATL60_envisat.nc",
    "/gpfswork/rech/yrf/commun/data_challenges/dc20a_osse/raw/dc_obs/2020a_SSH_mapping_NATL60_geosat2.nc",
    "/gpfswork/rech/yrf/commun/data_challenges/dc20a_osse/raw/dc_obs/2020a_SSH_mapping_NATL60_topex-poseidon_interleaved.nc"
]

files_nadir_dc21a = [
    "/gpfswork/rech/yrf/commun/data_challenges/dc21a_ose/test/train/dt_gulfstream_alg_phy_l3_20161201-20180131_285-315_23-53.nc",
    "/gpfswork/rech/yrf/commun/data_challenges/dc21a_ose/test/train/dt_gulfstream_h2g_phy_l3_20161201-20180131_285-315_23-53.nc",
    "/gpfswork/rech/yrf/commun/data_challenges/dc21a_ose/test/train/dt_gulfstream_j2g_phy_l3_20161201-20180131_285-315_23-53.nc",
    "/gpfswork/rech/yrf/commun/data_challenges/dc21a_ose/test/train/dt_gulfstream_j2n_phy_l3_20161201-20180131_285-315_23-53.nc",
    "/gpfswork/rech/yrf/commun/data_challenges/dc21a_ose/test/train/dt_gulfstream_j3_phy_l3_20161201-20180131_285-315_23-53.nc",
    "/gpfswork/rech/yrf/commun/data_challenges/dc21a_ose/test/train/dt_gulfstream_s3a_phy_l3_20161201-20180131_285-315_23-53.nc"
]


ds_nadir = xr.open_dataset(files_nadir_dc21a[0])

In [5]:
ds_nadir

<xarray.Dataset>
Dimensions:         (time: 404083)
Coordinates:
  * time            (time) datetime64[ns] 2016-12-01T08:49:09.733021952 ... 2...
    longitude       (time) float64 ...
    latitude        (time) float64 ...
Data variables:
    cycle           (time) float64 ...
    track           (time) float64 ...
    dac             (time) float32 ...
    lwe             (time) float32 ...
    mdt             (time) float32 ...
    ocean_tide      (time) float64 ...
    sla_filtered    (time) float32 ...
    sla_unfiltered  (time) float32 ...
Attributes: (12/44)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Swath
    comment:                         Sea surface height measured by altimeter...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    creator_email:                   servicedesk.cmems@mercator-ocean.eu
    ...                              ...
    summary:                         SSALTO/DUACS Delayed-Time Level-3 sea su...
    time_coverage_duration:          P23H16M26.458585S
    time_coverage_end:               2016-01-01T23:06:57Z
    time_coverage_resolution:        P1S
    time_coverage_start:             2015-12-31T23:50:30Z
    title:                           DT Altika Drifting Phase Global Ocean Al...

In [28]:
def preprocess_nadir_dc20a(da, variable="ssh_mod"):
        
    da = da.rename({variable: "ssh"})
    
    da = da.sel(
        # time=slice("2012-10-22", "2012-12-03"),
        drop=True
    ).compute()
    
    da["lon"] = transform_360_to_180(da["lon"])
    
    # da = where_slice(da, "lon", -64.975, -55.007)
    # da = where_slice(da, "lat", 33.025, 42.9917)
    
    da = da.drop_dims("cycle")
        
    return da

def preprocess_nadir_dc21a(da, variable="ssh_mod"):
        
    da = da.rename({variable: "ssh"})
    
    da = da.sel(
        # time=slice("2017-01-01", "2018-01-01"),
        drop=True
    ).compute()
    
    da = da.rename({"longitude": "lon", "latitude": "lat"})
    
    da["lon"] = transform_360_to_180(da["lon"])
    
    # da = where_slice(da, "lon", -64.975, -55.007)
    # da = where_slice(da, "lat", 33.025, 42.9917)
            
    return da

In [29]:
transform_360_to_180(np.asarray([295, 305]))

array([-65, -55])

In [30]:
preprocess_fn = ft.partial(preprocess_nadir_dc21a, variable="sla_unfiltered")

ds_nadir = xr.open_mfdataset(
    files_nadir_dc21a, 
    preprocess=preprocess_fn,
    combine="nested",
    engine="netcdf4",
    concat_dim="time"
)

ds_nadir = ds_nadir.sortby("time")

ds_nadir

<xarray.Dataset>
Dimensions:       (time: 1793663)
Coordinates:
  * time          (time) datetime64[ns] 2016-12-01T01:07:08.844252928 ... 201...
    lon           (time) float64 -50.5 -50.51 -50.52 ... -74.96 -74.97 -74.99
    lat           (time) float64 23.03 23.08 23.14 23.2 ... 25.21 25.27 25.33
Data variables:
    cycle         (time) float64 11.0 11.0 11.0 11.0 ... 193.0 193.0 193.0 193.0
    track         (time) float64 575.0 575.0 575.0 575.0 ... 83.0 83.0 83.0 83.0
    dac           (time) float32 -0.0565 -0.0569 -0.0571 ... -0.0842 -0.0847
    lwe           (time) float32 0.003 0.003 0.003 ... -0.005 -0.005 -0.005
    mdt           (time) float32 0.52 0.52 0.521 0.522 ... 0.621 0.622 0.623
    ocean_tide    (time) float64 0.0269 0.0288 0.0307 ... 0.302 0.3033 0.3046
    sla_filtered  (time) float32 0.101 0.103 0.105 0.109 ... 0.039 0.048 0.059
    ssh           (time) float32 0.1 0.098 0.104 0.115 ... -0.01 0.069 0.046
Attributes: (12/44)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Swath
    comment:                         Sea surface height measured by altimeter...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    creator_email:                   servicedesk.cmems@mercator-ocean.eu
    ...                              ...
    summary:                         SSALTO/DUACS Delayed-Time Level-3 sea su...
    time_coverage_duration:          P23H16M26.458585S
    time_coverage_end:               2016-01-01T23:06:57Z
    time_coverage_resolution:        P1S
    time_coverage_start:             2015-12-31T23:50:30Z
    title:                           DT Altika Drifting Phase Global Ocean Al...

In [31]:
ds_nadir.lon.min(), ds_nadir.lon.max(), ds_nadir.lat.min(), ds_nadir.lat.max()

(<xarray.DataArray 'lon' ()>
 array(-74.999987),
 <xarray.DataArray 'lon' ()>
 array(-45.000005),
 <xarray.DataArray 'lat' ()>
 array(23.000021),
 <xarray.DataArray 'lat' ()>
 array(52.999899))

In [18]:
preprocess_fn = ft.partial(preprocess_nadir_dc20a, variable="ssh_model")

ds_nadir = xr.open_mfdataset(
    files_nadir_dc20a, 
    preprocess=preprocess_fn,
    combine="nested",
    engine="netcdf4",
    concat_dim="time"
)

ds_nadir = ds_nadir.sortby("time")

ds_nadir

<xarray.Dataset>
Dimensions:      (time: 205232)
Coordinates:
  * time         (time) datetime64[ns] 2012-10-01T00:45:50.676369 ... 2013-09...
Data variables:
    lon          (time) float64 -57.81 -57.83 -57.85 ... -58.29 -58.31 -58.33
    lat          (time) float64 42.96 42.9 42.84 42.78 ... 42.88 42.94 43.0
    x_al         (time) float32 1.257e+07 1.257e+07 ... 1.689e+07 1.689e+07
    model_index  (time) float32 1.0 1.0 1.0 1.0 ... 8.75e+03 8.75e+03 8.75e+03
    ssh_obs      (time) float64 -0.0045 -0.0243 -0.0236 ... 0.2423 0.2093 0.1756
    ssh          (time) float64 -0.0028 -0.0097 -0.0153 ... 0.244 0.2186 0.1897
    nadir_err    (time) float64 -0.0017 -0.0145 -0.0082 ... -0.0093 -0.0141
Attributes: (12/26)
    description:               SWOT fixed grid
    corresponding_grid:        
    title:                     Altimeter like data simulated by SWOT simulator
    keywords:                  check keywords
    Conventions:               CF-1.6
    summary:                   SWOT grid data produced
    ...                        ...
    geospatial_lon_units:      degrees_east
    project:                   SWOT
    date_created:              2018-10-12T12:39:50Z
    date_modified:             2018-10-12T12:39:50Z
    keywords_vocabulary:       NASA
    references:                Gaultier, L., C. Ubelmann, and L.-L. Fu, 2016:...

In [15]:
# %matplotlib inline

# fig, ax = plt.subplots()

# sub_ds = ds_nadir.isel(time=slice(0,None))
# pts = ax.scatter(sub_ds.lon, sub_ds.lat, c=sub_ds.ssh, s=0.1)
# ax.set(
#     xlabel="Longitude",
#     ylabel="Latitude",
# )

# plt.colorbar(pts, label="Sea Surface Height [m]")
# plt.tight_layout()
# plt.show()

## Data Structure

## Gridding

In [9]:
# !ls /gpfswork/rech/yrf/commun/data_challenges/dc20a_osse/staging/natl60/
!ls /gpfswork/rech/yrf/commun/data_challenges/dc21a_ose/test/results

OSE_ssh_mapping_4dvarNet_2022.nc  OSE_ssh_mapping_DUACS.nc
OSE_ssh_mapping_4dvarNet.nc	  OSE_ssh_mapping_DYMOST.nc
OSE_ssh_mapping_BASELINE.nc	  OSE_ssh_mapping_MIOST.nc
OSE_ssh_mapping_BFN.nc


In [10]:
file_natl60 = "/gpfswork/rech/yrf/commun/data_challenges/dc20a_osse/staging/natl60/NATL60-CJM165_GULFSTREAM_ssh_y2013.1y.nc"
file_DUACS = "/gpfswork/rech/yrf/commun/data_challenges/dc21a_ose/test/results/OSE_ssh_mapping_DUACS.nc"


In [11]:
def open_natl60_reference(file, variable="gssh"):
    da = xr.open_dataset(file, decode_times=False)
    da["time"] = pd.to_datetime(da.time)
    da = da.sortby("time")
    da["lon"] = transform_360_to_180(da["lon"])
    da = da.sel(
        time=slice("2012-10-22", "2012-12-03"),
        lon=slice(-64.975, -55.007),
        lat=slice(33.025, 42.9917),
        drop=True
    )
    da = da.rename({variable: "ssh"})
    return da


def open_ose_reference(file, variable="gssh"):
    da = xr.open_dataset(file, decode_times=True)
    da = da.sortby("time")
    da["lon"] = transform_360_to_180(da["lon"])
    da = da.sel(
        time=slice("2017-01-01", "2017-03-01"),
        lon=slice(-64.975, -55.007),
        lat=slice(33.025, 42.9917),
        drop=True
    )
    da = da.rename({variable: "ssh"})
    return da

In [12]:
# ds_natl60 = open_ssh_reference(file_natl60, "ssh")
ds_duacs = open_ose_reference(file_DUACS, "ssh")
ds_duacs

<xarray.Dataset>
Dimensions:  (lat: 40, lon: 40, time: 60)
Coordinates:
  * lat      (lat) float64 33.12 33.38 33.62 33.88 ... 42.12 42.38 42.62 42.88
  * lon      (lon) float64 -64.88 -64.62 -64.38 -64.12 ... -55.62 -55.38 -55.12
  * time     (time) datetime64[ns] 2017-01-01 2017-01-02 ... 2017-03-01
Data variables:
    ssh      (time, lat, lon) float64 ...
Attributes:
    FileType:      GRID_DOTS
    OriginalName:  dt_upd_global_merged_msla_h_20170101_20170101_20190823.nc
    CreatedBy:     ballarm@node036.sis.cnes.fr
    CreatedOn:     23-AUG-2019 11:21:19:000000
    title:         SSALTO/DUACS - DT MSLA - Merged Product - Up-to-date Globa...
    history:       2019/08/23 11:21:19 ballarm@node036.sis.cnes.fr Import dep...

In [13]:
import pyinterp



In [14]:
from oceanbench._src.geoprocessing.gridding import coord_based_to_grid

In [15]:
ds_nadir_gridded = coord_based_to_grid(
    ds_nadir, 
    ds_duacs,
    data_vars=["ssh"], 
    t_res=pd.to_timedelta(12, unit="hour")
)
# np.isfinite(ds_nadir_gridded.ssh.isel(time=6)).plot.imshow()

In [16]:
# import holoviews as hv
# hv.extension("matplotlib")


# variable = "ssh" # "vort_r" # "ke" #  
# cmap = "viridis" # "RdBu_r" # "YlGnBu_r" #
# field_name = "DUACS"

# ssh_ds = xr.Dataset({
#     field_name: ds_duacs[variable],
#     "NADIR": np.isfinite(ds_nadir_gridded[variable]),
# })


# to_plot_ds = ssh_ds.transpose("time", "lat", "lon")#.isel(time=slice(25, 55, 1))

# clim = (
#     to_plot_ds[[field_name, "NADIR"]].to_array().pipe(lambda da: (da.quantile(0.005).item(), da.quantile(0.995).item()))
# )

# images = hv.Layout([
#     hv.Dataset(to_plot_ds)
#     .to(hv.QuadMesh, ["lon", "lat"], v).relabel(v)
#     .options(cmap=cmap, clim=clim)
#     for v in to_plot_ds]
# ).cols(2).opts(sublabel_format="")

# hv.output(images, holomap="gif", fps=2, dpi=125)

## Segments

## PSD Analysis